<a href="https://colab.research.google.com/github/abhi147p/Abhishek_INFO5731_Spring2023/blob/main/In_class_exercise_04_03282023_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here

# getting the data
# You code here (Please add comments in the code):

import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

def get_soup(url):
    """
        params1: url (contains the url of google scholar page)
        return: soup (fetching the url page data and then further converted to html parser)
    """
  # headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    headers = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36')}
               # 'X-User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 FKUA/website/42/website/Desktop')}
    try:
        # requesting for data using requests with url and headers for authentication
        data = requests.get(url, headers=headers)
        # print(f"Extracted the data with response as {data.status_code}")
        if data.status_code != 200:
            raise Exception(f"Failed to fetch data: {data.status_code} and text as {data.text}")
    except Exception as ex:
        print(f"Exception occurred as {ex}")
        return None
    soup = BeautifulSoup(data.text, "html.parser")
    return soup

def fetch_data_web(pages):

  final_data = []
  for i in range(1, pages+1):
    url = f"https://www.flipkart.com/search?q=Watches&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={i}"
    soup = get_soup(url)
    # print(soup.prettify())
    products_data = soup.findAll("div", attrs={"class": "_1AtVbE col-12-12"})
    for prod_data in products_data:
      sub_prod_data = prod_data.findAll("div", attrs={"class": "_1xHGtK _373qXS"})
      for data in sub_prod_data:
        prod_url = "https://www.flipkart.com" + data.find("a", attrs={"class": "_2UzuFa"}).get("href")
        brand = data.find("div", attrs={"class": "_2WkVRV"}).text.strip()
        title = data.find("a", attrs={"class": "IRpwTa"}).get("title")
        new_price = data.find("div", attrs={"class": "_30jeq3"})
        old_price = data.find("div", attrs={"class": "_3I9_wc"})
        if new_price:
          new_price = new_price.text.strip()
        else:
          new_price = "No"
        if old_price:
          old_price = old_price.text.strip()
        else:
          old_price = "No"
        # discount = data.find("div", attrs={"class": "_3Ay6Sb"}).text.strip()
        # free_delivery = data.find("div", attrs={"class": "_2Tpdn3"})
        # if free_delivery:
        #   free_delivery = free_delivery.text.strip()
        # else:
        #   free_delivery = "No free Delivery"
        soup_data = get_soup(prod_url)
        # print(prod_url)
        # print(soup_data)
        # break
        # rating = "0"
        # if soup_data:
        if soup_data:
          rating = soup_data.find("div", attrs={"class": "_3LWZlK _3uSWvT"})
          if rating:
            rating = rating.text
          else:
            rating = "No rating found"
          reviews = soup_data.findAll("div", attrs={"class": "_6K-7Co"})
          for x in reviews:
            final_data.append([title, prod_url, brand, new_price, old_price, rating, x.text])
        else:
          rating = "No rating reviews found"
        # final_data.append([title, prod_url, brand, new_price, old_price, rating, review])
      time.sleep(1)
    print(f"Processed {i} pages")
  
  df = pd.DataFrame(final_data, columns=["Title", "url", "brand", "new_price", "old_price", "rating", "review"])
  return df

df = fetch_data_web(int(input("Enter no of pages: ")))
df.head()

Enter no of pages: 3
Processed 1 pages
Processed 2 pages
Processed 3 pages


,Title,url,brand,new_price,old_price,rating,review
0,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Espoir,₹419,"₹2,995",3.9,This product is so good but I think packaging ...
1,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Espoir,₹419,"₹2,995",3.9,so beautiful watches
2,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Espoir,₹419,"₹2,995",3.9,Nice 🙂🙂
3,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,GLENVIT-X,₹424,"₹1,499",3.9,Worth products the footstep is wrong the heart...
4,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,GLENVIT-X,₹424,"₹1,499",3.9,Good


In [ ]:
df = df[df['old_price'] != "No"]

In [ ]:
# converting str to numeric

df['new_price'] = df['new_price'].str.replace(',', '').str[1:]
df['old_price'] = df['old_price'].str.replace(',', '').str[1:]

In [ ]:
df['new_price'] = df['new_price'].astype(int)
df['old_price'] = df['old_price'].astype(int)
df['rating'] = df['rating'].astype(float)

df.head()

,Title,url,brand,new_price,old_price,rating,review
0,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Espoir,419,2995,3.9,This product is so good but I think packaging ...
1,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Espoir,419,2995,3.9,so beautiful watches
2,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Espoir,419,2995,3.9,Nice 🙂🙂
3,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,GLENVIT-X,424,1499,3.9,Worth products the footstep is wrong the heart...
4,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,GLENVIT-X,424,1499,3.9,Good


In [ ]:
df = df[['Title', 'url', 'review']]
df.head()

,Title,url,review
0,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,This product is so good but I think packaging ...
1,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,so beautiful watches
2,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Nice 🙂🙂
3,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,Worth products the footstep is wrong the heart...
4,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,Good


In [ ]:
df['review'].isna().sum()

0

In [ ]:
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df.head()

,Title,url,review
0,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,This product is so good but I think packaging ...
1,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,so beautiful watches
2,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Nice 🙂🙂
3,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,Worth products the footstep is wrong the heart...
4,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,Good


In [ ]:
import re

df['clean_Data'] = df['review'].map(lambda x: re.sub('[,\.!?]', '', x))
df['clean_Data'] = df['clean_Data'].map(lambda x: x.lower())
df.head()

,Title,url,review,clean_Data
0,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,This product is so good but I think packaging ...,this product is so good but i think packaging ...
1,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,so beautiful watches,so beautiful watches
2,Chronograph Pattern Combo of 3 watches Analog ...,https://www.flipkart.com/espoir-combo-es109-ba...,Nice 🙂🙂,nice 🙂🙂
3,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,Worth products the footstep is wrong the heart...,worth products the footstep is wrong the heart...
4,Digital Dial Waterproof LED Display Kids Watch...,https://www.flipkart.com/glenvit-x-kids-digita...,Good,good


In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 35.8 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [ ]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

review_data = df['review'].values.tolist()
review_data_words = list(sent_to_words(review_data))
# remove stop words
review_data_words = remove_stopwords(review_data_words)
print(review_data_words[:1])

[['product', 'good', 'think', 'packaging', 'better', 'still', 'watches', 'worth', 'go', 'love', 'good', 'words', 'watches', 'recommend', 'buy', 'watches', 'really', 'worth']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
bigram = gensim.models.Phrases(review_data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[review_data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

review_data_words_bigrams = make_bigrams(review_data_words)

# Do lemmatization keeping only noun, adj, vb, adv
review_data_lemmatized = lemmatization(review_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(review_data_lemmatized[:1])

[['product', 'good', 'think', 'packaging', 'well', 'still', 'watch', 'worth', 'go', 'love', 'good', 'word', 'watch', 'recommend', 'buy', 'watch', 'really', 'worth']]


In [ ]:
review_id2word = corpora.Dictionary(review_data_lemmatized) # corpus dictionary
review_texts = review_data_lemmatized
review_corpus = [review_id2word.doc2bow(text) for text in review_texts] # doc term matrix
print(review_corpus[:1])
[[(review_id2word[i], f) for i, f in c] for c in review_corpus[:1]]

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 3), (11, 1), (12, 1), (13, 2)]]


[[('buy', 1),
  ('go', 1),
  ('good', 2),
  ('love', 1),
  ('packaging', 1),
  ('product', 1),
  ('really', 1),
  ('recommend', 1),
  ('still', 1),
  ('think', 1),
  ('watch', 3),
  ('well', 1),
  ('word', 1),
  ('worth', 2)]]

In [ ]:
review_id2word[0]

'buy'

<b> LDA MODEL </b>

In [ ]:
# Sample model for lda generating total of 20 topics
from pprint import pprint
lda_model = gensim.models.ldamodel.LdaModel(corpus=review_corpus,
                                           id2word=review_id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[review_corpus]

[(0,
  '0.058*"class" + 0.058*"first" + 0.038*"thank" + 0.018*"dial" + '
  '0.017*"thanx" + 0.017*"golden" + 0.017*"large" + 0.017*"quantity" + '
  '0.002*"wear" + 0.002*"good"'),
 (1,
  '0.101*"excellent" + 0.075*"watch" + 0.074*"expect" + 0.062*"dial" + '
  '0.060*"strap" + 0.052*"belt" + 0.050*"quality" + 0.043*"look" + 0.039*"big" '
  '+ 0.031*"deal"'),
 (2,
  '0.002*"steel" + 0.002*"stylish" + 0.002*"hindu" + 0.002*"mean" + '
  '0.002*"new" + 0.002*"religion" + 0.002*"specially" + 0.002*"early" + '
  '0.002*"wai" + 0.002*"year"'),
 (3,
  '0.344*"watch" + 0.083*"buy" + 0.073*"good" + 0.062*"time" + 0.037*"cost" + '
  '0.031*"month" + 0.027*"look" + 0.026*"purchase" + 0.026*"fully" + '
  '0.023*"much"'),
 (4,
  '0.260*"awesome" + 0.066*"rate" + 0.026*"wrong" + 0.014*"footstep" + '
  '0.014*"heartbeat" + 0.001*"yet" + 0.001*"display" + 0.001*"statement" + '
  '0.001*"future" + 0.001*"work"'),
 (5,
  '0.387*"money" + 0.250*"value" + 0.054*"super" + 0.044*"good" + '
  '0.028*"product" 

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=review_data_lemmatized, dictionary=review_id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.41090161889315996


In [ ]:
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [ ]:
# calculating model coherence value for various number of topics to get the best k number
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
start, limit, step=2, 20, 2
model_list, coherence_values = compute_coherence_values(dictionary=review_id2word, corpus=review_corpus, texts=review_data_lemmatized, start=start, limit=limit, step=step)
coherence_values_all = []
for m, cv in zip(range(start, limit, step), coherence_values):
    coherence_values_all.append(round(cv,4))
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.4324
Num Topics = 4  has Coherence Value of 0.3827
Num Topics = 6  has Coherence Value of 0.4073
Num Topics = 8  has Coherence Value of 0.3813
Num Topics = 10  has Coherence Value of 0.4007
Num Topics = 12  has Coherence Value of 0.3788
Num Topics = 14  has Coherence Value of 0.3605
Num Topics = 16  has Coherence Value of 0.3921
Num Topics = 18  has Coherence Value of 0.435


In [ ]:
sum(coherence_values_all)/len(coherence_values_all)

0.3967555555555556

As we can see the values are first decreased and then increasing. Hence taking the value, which is near to mean of all coherence values. Taking 16 topics with value 0.39. 

In [ ]:
optimal_model = model_list[7] # getting the topics=16 model, which is in 7th index of our model and getting the 16 topics.
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.163*"warranty" + 0.137*"month" + 0.013*"time" + 0.010*"seller" + '
  '0.010*"cargo" + 0.010*"night" + 0.010*"neat" + 0.010*"changer" + '
  '0.010*"useful" + 0.010*"auto"'),
 (1,
  '0.186*"money" + 0.133*"value" + 0.055*"expect" + 0.053*"excellent" + '
  '0.038*"look" + 0.036*"quality" + 0.033*"watch" + 0.032*"classy" + '
  '0.030*"design" + 0.029*"big"'),
 (2,
  '0.548*"nice" + 0.193*"product" + 0.083*"look" + 0.027*"quality" + '
  '0.011*"choice" + 0.008*"productb" + 0.006*"diesel" + 0.006*"bill" + '
  '0.000*"packing" + 0.000*"wacth"'),
 (3,
  '0.099*"look" + 0.091*"watch" + 0.078*"colour" + 0.052*"pretty" + '
  '0.046*"wise" + 0.046*"wear" + 0.045*"buy" + 0.038*"overall" + 0.036*"good" '
  '+ 0.032*"excellent"'),
 (4,
  '0.102*"watch" + 0.085*"good" + 0.076*"look" + 0.061*"get" + 0.051*"well" + '
  '0.044*"worth" + 0.039*"range" + 0.037*"show" + 0.037*"use" + 0.037*"even"'),
 (5,
  '0.123*"simple" + 0.087*"super" + 0.043*"money" + 0.037*"short" + '
  '0.034*"wish" + 0.031*

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here

from gensim.models import LsiModel

lsamodel = LsiModel(review_corpus, num_topics=10, id2word = review_id2word)  # train model
print(lsamodel.print_topics(num_topics=10, num_words=10))


[(0, '0.564*"watch" + 0.408*"product" + 0.385*"good" + 0.323*"nice" + 0.227*"look" + 0.203*"quality" + 0.120*"thank" + 0.111*"love" + 0.100*"flipkart" + 0.098*"money"'), (1, '0.698*"watch" + -0.584*"product" + -0.200*"good" + -0.182*"nice" + -0.078*"flipkart" + -0.076*"really" + -0.072*"thank" + -0.068*"money" + 0.068*"look" + 0.051*"dial"'), (2, '0.702*"good" + -0.479*"product" + 0.293*"quality" + -0.212*"watch" + 0.120*"money" + -0.098*"nice" + 0.096*"fast" + 0.094*"price" + 0.082*"look" + -0.078*"give"'), (3, '-0.847*"nice" + 0.236*"product" + 0.177*"good" + 0.131*"love" + -0.125*"quality" + 0.116*"go" + 0.109*"really" + -0.097*"money" + 0.093*"design" + 0.091*"give"'), (4, '-0.720*"look" + 0.322*"good" + 0.233*"watch" + -0.160*"thank" + -0.140*"quality" + -0.138*"strap" + -0.115*"money" + -0.113*"great" + -0.103*"delivery" + -0.101*"excellent"'), (5, '0.567*"thank" + 0.372*"flipkart" + 0.341*"love" + -0.241*"product" + 0.214*"much" + 0.155*"amazing" + 0.151*"beautiful" + -0.145*"lo

In [ ]:
coherencemodel = CoherenceModel(model=lsamodel, texts=review_data_lemmatized, dictionary=review_id2word, coherence='c_v')
print(coherencemodel.get_coherence())

0.35844712476622786


In [ ]:
def compute_lsa_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        lsamodel = LsiModel(corpus, num_topics=num_topics, id2word = dictionary)  # train model
        model_list.append(lsamodel)
        coherencemodel = CoherenceModel(model=lsamodel, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

start, limit, step=2, 20, 2
model_lsa_list, coherence_values = compute_lsa_coherence_values(dictionary=review_id2word, corpus=review_corpus, texts=review_data_lemmatized, start=start, limit=limit, step=step)
coherence_values_all = []
for m, cv in zip(range(start, limit, step), coherence_values):
    coherence_values_all.append(round(cv,4))
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.3746
Num Topics = 4  has Coherence Value of 0.3563
Num Topics = 6  has Coherence Value of 0.4036
Num Topics = 8  has Coherence Value of 0.3238
Num Topics = 10  has Coherence Value of 0.3534
Num Topics = 12  has Coherence Value of 0.328
Num Topics = 14  has Coherence Value of 0.3653
Num Topics = 16  has Coherence Value of 0.3095
Num Topics = 18  has Coherence Value of 0.3047


In [ ]:
coherence_values_all

[0.3746, 0.3563, 0.4036, 0.3238, 0.3534, 0.328, 0.3653, 0.3095, 0.3047]

In [ ]:
sum(coherence_values_all)/len(coherence_values_all)

0.34657777777777776

As we can see the values are first decreased and then increasing, which is in unorganised way. Hence taking the value, which is near to mean of all coherence values. Taking 10 topics with value 0.35. 

In [ ]:
optimal_model = model_list[4] # getting the topics=10 model, which is in 5th index of our model and getting the 10 topics.
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.096*"good" + 0.076*"love" + 0.060*"product" + 0.058*"quality" + '
  '0.046*"really" + 0.041*"great" + 0.041*"design" + 0.029*"much" + '
  '0.028*"price" + 0.027*"also"'),
 (1,
  '0.058*"happy" + 0.053*"item" + 0.046*"excellent" + 0.043*"expect" + '
  '0.039*"delivery" + 0.035*"beautiful" + 0.031*"strap" + 0.030*"quality" + '
  '0.029*"thank" + 0.027*"much"'),
 (2,
  '0.047*"perfect" + 0.042*"use" + 0.041*"keep" + 0.040*"thing" + '
  '0.029*"amazing" + 0.026*"sleek" + 0.025*"know" + 0.023*"get" + 0.022*"day" '
  '+ 0.021*"fantastic"'),
 (3,
  '0.085*"watch" + 0.052*"look" + 0.045*"colour" + 0.044*"product" + '
  '0.038*"recommend" + 0.035*"pretty" + 0.033*"wrist" + 0.030*"wise" + '
  '0.029*"good" + 0.028*"buy"'),
 (4,
  '0.131*"watch" + 0.067*"product" + 0.063*"love" + 0.057*"awesome" + '
  '0.045*"beautiful" + 0.042*"much" + 0.038*"give" + 0.035*"well" + 0.032*"go" '
  '+ 0.026*"guy"'),
 (5,
  '0.262*"nice" + 0.045*"pic" + 0.038*"gifted" + 0.025*"choice" + '
  '0.024*"wacth"

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here

import lda2vec




ModuleNotFoundError: ignored

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here

%%capture
!pip install bertopic

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(df['review'].tolist())

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



